<a href="https://colab.research.google.com/github/AndresWV/AndresWV.github.io/blob/main/ProyectoWisdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Librerias usadas**

In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

**1) Obtain**


1.1 Crear archivo sin puntos y comas para despues crear el df




In [12]:
!sed 's/;//' 'WISDM_ar_v1.1_raw.txt'  > 'wisdmHar.nosemi'


sed: can't read WISDM_ar_v1.1_raw.txt: No such file or directory


1.2 Definir cabecera para generar el df con los datos



In [ ]:
columns = ['user','activity','timestamp','x-axis','y-axis','z-axis']
wisdmHar = pd.read_csv('wisdmHar.nosemi',header=None,names=columns,sep=',').dropna()


,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


**Entrega 02 - SCRUB**



Eliminar filas con datos faltantes


In [ ]:
wisdmHar.dropna() 

Agregar al DataFrame los nombres de las columnas(Esta en el 1.2)



Visualizar las 5 primeras filas del DataFrame (.head())


In [ ]:
wisdmHar.head()

Buscar datos nulos y eliminarlos si es que existen

Visualizar los tipo de datos a los que pertenece cada columna (.info())

In [ ]:
wisdmHar.info()

Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación